In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
import psycopg2 as pg
import numpy as np
import pandas as pd
import datetime
import logging

# connect to volunteer database using 2017-08-05 data dump
connection = pg.connect(database="sidewalk_turker", user="sidewalk", password="sidewalk", host="localhost", port="5432" )

In [3]:
turker_accuracy = pd.read_csv('../../data/raw/turker.csv')
turker_accuracy = turker_accuracy[turker_accuracy['n.workers'] == 1]
turker_accuracy

/Users/johann/Code/sidewalk/interaction/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,condition.id,worker1,worker2,worker3,worker4,worker5,n.workers,worker.thresh,binary,remove.low.severity,...,granularity,label.type,true.pos,false.pos,true.neg,false.neg,precision,recall,specificity,f.measure
0,70,A3PPRVK6XK6GP5,null,null,null,null,1,1,True,False,...,5_meter,Problem,20,10,202,24,0.6666666666666666,0.45454545454545453,0.9528301886792453,0.5405405405405405
1,70,A3PPRVK6XK6GP5,null,null,null,null,1,1,True,False,...,10_meter,Problem,19,6,85,20,0.76,0.48717948717948717,0.9340659340659341,0.59375
2,70,A3PPRVK6XK6GP5,null,null,null,null,1,1,True,False,...,street,Problem,10,0,0,7,1,0.5882352941176471,null,0.7407407407407407
3,72,A1TNQU9L5L62PL,null,null,null,null,1,1,True,False,...,5_meter,Problem,21,70,149,8,0.23076923076923078,0.7241379310344828,0.680365296803653,0.35000000000000003
4,72,A1TNQU9L5L62PL,null,null,null,null,1,1,True,False,...,10_meter,Problem,19,44,59,5,0.30158730158730157,0.7916666666666666,0.5728155339805825,0.43678160919540227
5,72,A1TNQU9L5L62PL,null,null,null,null,1,1,True,False,...,street,Problem,11,0,1,0,1,1,1,1
6,73,null,null,null,null,null,1,1,True,False,...,5_meter,Problem,0,0,202,48,null,0,1,null
7,73,null,null,null,null,null,1,1,True,False,...,10_meter,Problem,0,0,93,38,null,0,1,null
8,73,null,null,null,null,null,1,1,True,False,...,street,Problem,0,0,1,13,null,0,1,null
9,74,A20BMZQJS92QY2,null,null,null,null,1,1,True,False,...,5_meter,Problem,26,39,92,5,0.4,0.8387096774193549,0.7022900763358778,0.5416666666666666


In [4]:
selected_users = turker_accuracy[['condition.id', 'worker1']]
selected_users = selected_users.drop_duplicates()
len(selected_users)

275

In [5]:
def get_assignment_id(condition_id, turker_id):
    return pd.read_sql(
    '''
        SELECT amt_assignment_id
        FROM amt_assignment
        WHERE condition_id='%(condition_id)s' AND turker_id='%(turker_id)s'
    ''' % locals(), connection)

def get_route_ids(condition_id, turker_id):
    return pd.read_sql(
    '''
        SELECT DISTINCT route_id
        FROM amt_assignment
        WHERE condition_id='%(condition_id)s' AND turker_id='%(turker_id)s'
    ''' % locals(), connection)

def get_street_edges(route_id):
    return pd.read_sql(
    '''
        SELECT current_street_edge_id
        FROM route_street
        WHERE route_id='%(route_id)s'
    ''' % locals(), connection)

def get_audit_tasks(assignment_id, street_edge):
    return pd.read_sql(
    '''
        SELECT audit_task_id
        FROM audit_task
        WHERE amt_assignment_id='%(assignment_id)s' AND street_edge_id='%(street_edge)s'
    ''' % locals(), connection)

def get_audit_interactions(audit_task_id):
    return pd.read_sql(
    '''
        SELECT *
        FROM audit_task_interaction
        WHERE audit_task_id='%(audit_task_id)s'
    ''' % locals(), connection)

In [6]:
action_types = pd.read_sql(
    '''
        SELECT DISTINCT action
        FROM audit_task_interaction
    ''', connection)
action_types = action_types['action'].tolist()
action_types

['ModeSwitch_Walk',
 'KeyboardShortcut_ModeSwitch_CurbRamp',
 'ViewControl_DoubleClick',
 'KeyboardShortcut_ClickOk',
 'KeyUp',
 'LabelBeforeJump_Jump',
 'LabelBeforeJump_Blink',
 'LowLevelEvent_mouseout',
 'LowLevelEvent_keydown',
 'KeyDown',
 'Redo_AddLabel',
 'Click_Redo',
 'ContextMenu_RadioChange',
 'KeyboardShortcut_ModeSwitch_NoSidewalk',
 'TaskStart',
 'Click_ZoomOut',
 'Click_Subcategory_Other',
 'LowLevelEvent_mousedown',
 'Onboarding_End',
 'TaskSubmit',
 'LowLevelEvent_mouseover',
 'ContextMenu_TextBoxFocus',
 'ModeSwitch_CurbRamp',
 'KeyboardShortcut_Severity_1',
 'Onboarding_Start',
 'ModeSwitch_Occlusion',
 'KeyboardShortcut_ZoomIn',
 'PopUpShow_GSVLabelDisappear',
 'Click_ModeSwitch_NoCurbRamp',
 'LabelingCanvas_MouseUp',
 'Onboarding_Transition',
 'ModalComment_ClickOK',
 'ExplainThis_Click',
 'ContextMenu_Open',
 'Click_ModeSwitch_Obstacle',
 'KeyboardShortcut_Severity_3',
 'Click_ZoomIn',
 'Click_Subcategory_Occlusion',
 'ContextMenu_CheckboxChange',
 'ContextMenu_Cl

In [7]:
def get_action_counts(audit_inter):
    features = {}
    
    for action in action_types:
        a = audit_inter[audit_inter['action'] == action]
        g = a.groupby('gsv_panorama_id').size()
        
        total = len(a)
        if len(a) == 0:
            mean = 0
            std = 0
        else: # TODO: handle stddev NaNs
            mean = g.mean()
            std = g.std()
                
            
        features[action + '_total'] = total
        features[action + '_per_pan_mean'] = mean
        features[action + '_per_pan_std'] = std
    
    return features

In [8]:
features_per_action = ['_total', '_per_pan_mean', '_per_pan_std']
action_features = [(action + feature) for action in action_types for feature in features_per_action ]
action_features.sort()
action_features

['Click_LabelDelete_per_pan_mean',
 'Click_LabelDelete_per_pan_std',
 'Click_LabelDelete_total',
 'Click_ModeSwitch_CurbRamp_per_pan_mean',
 'Click_ModeSwitch_CurbRamp_per_pan_std',
 'Click_ModeSwitch_CurbRamp_total',
 'Click_ModeSwitch_NoCurbRamp_per_pan_mean',
 'Click_ModeSwitch_NoCurbRamp_per_pan_std',
 'Click_ModeSwitch_NoCurbRamp_total',
 'Click_ModeSwitch_Obstacle_per_pan_mean',
 'Click_ModeSwitch_Obstacle_per_pan_std',
 'Click_ModeSwitch_Obstacle_total',
 'Click_ModeSwitch_Other_per_pan_mean',
 'Click_ModeSwitch_Other_per_pan_std',
 'Click_ModeSwitch_Other_total',
 'Click_ModeSwitch_SurfaceProblem_per_pan_mean',
 'Click_ModeSwitch_SurfaceProblem_per_pan_std',
 'Click_ModeSwitch_SurfaceProblem_total',
 'Click_ModeSwitch_Walk_per_pan_mean',
 'Click_ModeSwitch_Walk_per_pan_std',
 'Click_ModeSwitch_Walk_total',
 'Click_OpenSkipWindow_per_pan_mean',
 'Click_OpenSkipWindow_per_pan_std',
 'Click_OpenSkipWindow_total',
 'Click_Redo_per_pan_mean',
 'Click_Redo_per_pan_std',
 'Click_Redo_

In [9]:
def get_features(condition_id, worker_id):
    all_audit_interactions = None
    logging.debug('Condition_id, worker_id: ' + str(condition_id) + ', ' + str(worker_id))
    
    feature_df = pd.DataFrame(columns=(['condition_id', 'worker_id'] + action_features))
    
    assignment_id = get_assignment_id(condition_id, worker_id)
    if len(assignment_id) == 0:
        # TODO: look into skip
        logging.warning('SKIPPING condition_id, worker_id: ' + str(condition_id) + ', ' + str(worker_id))
        return
    
    # TODO: handle multiple assignments per turker
    assignment_id = assignment_id['amt_assignment_id'][0]
    routes = get_route_ids(condition_id, worker_id)
    
    logging.debug('Assignment id: ' + str(assignment_id))

    for i, route in routes[:3].iterrows():
        route_number = route['route_id']
        street_edges = get_street_edges(route_number)
        
        logging.debug('  Route_id: ' + str(route_number))
        
        for j, street_edge_series in street_edges.iterrows():
            street_edge = street_edge_series['current_street_edge_id']
            audit_tasks = get_audit_tasks(assignment_id, street_edge)
            
            logging.debug('    Street edge: ' + str(street_edge))
            logging.debug('    Num audit_tasks: ' + str(len(audit_tasks)))
            
            for k, audit_task in audit_tasks.iterrows():
                audit_task_id = audit_task['audit_task_id']
                audit_interactions = get_audit_interactions(audit_task_id)
                
                logging.debug('      Audit task id: ' + str(audit_task_id))
                logging.debug('      Num interactions: ' + str(len(audit_interactions)))

                if all_audit_interactions is None:
                    all_audit_interactions = audit_interactions
                else:
                    all_audit_interactions = all_audit_interactions.append(audit_interactions)
                
    if all_audit_interactions is None:
        logging.warning('SKIPPING condition_id, worker_id because no iteraction: ' + str(condition_id) + ', ' + str(worker_id))
        return
    action_counts = get_action_counts(all_audit_interactions)
    feature_df.loc[len(feature_df)] = [condition_id, worker_id] + [action_counts[action_feature] for action_feature in action_features]
    return feature_df

In [10]:
features = None

for i, user in selected_users.iterrows():
    
    f = get_features(user['condition.id'], user['worker1'])
    if features is None:
        features = f
    else:
        features = features.append(f)

features.reset_index(inplace=True)

DEBUG:root:Condition_id, worker_id: 70, A3PPRVK6XK6GP5
DEBUG:root:Assignment id: 3007
DEBUG:root:  Route_id: 3
DEBUG:root:    Street edge: 12694
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 12693
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 12691
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 20591
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 12695
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 12696
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:  Route_id: 1
DEBUG:root:    Street edge: 0
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 20902
DEBUG:root:      Num interactions: 18461
DEBUG:root:    Street edge: 83
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 21026
DEBUG:root:      Num interactions: 321
DEBUG:root:    Street edge: 9235
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 21028
DEBUG:root:      Num interactions: 7116
DEBUG:root:    Street edge

DEBUG:root:    Street edge: 14657
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 13768
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 17398
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:  Route_id: 190
DEBUG:root:    Street edge: 13762
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 18837
DEBUG:root:      Num interactions: 18418
DEBUG:root:    Street edge: 13758
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 18847
DEBUG:root:      Num interactions: 11657
DEBUG:root:    Street edge: 22876
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 18855
DEBUG:root:      Num interactions: 10318
DEBUG:root:Condition_id, worker_id: 78, A209V1OUPBH5XP
DEBUG:root:Assignment id: 2785
DEBUG:root:  Route_id: 149
DEBUG:root:    Street edge: 15149
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 15150
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 20304
DEBUG:root:      Num interactions: 4291
D

DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 24131
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 24130
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 25689
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 25693
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 17530
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:  Route_id: 225
DEBUG:root:    Street edge: 25695
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 25696
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 17529
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 25680
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 21047
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 17823
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 17824
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 101
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 25856
DEBUG:root:    Num audi

DEBUG:root:      Num interactions: 321
DEBUG:root:    Street edge: 12340
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 12341
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 17138
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:  Route_id: 110
DEBUG:root:    Street edge: 17138
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 17137
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 14782
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 615
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:  Route_id: 111
DEBUG:root:    Street edge: 615
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 23125
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:Condition_id, worker_id: 92, A1M0M61W98XPG0
DEBUG:root:Assignment id: 785
DEBUG:root:  Route_id: 68
DEBUG:root:    Street edge: 11373
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 15239
DEBUG:root:      Num interactions: 7734
DEBUG:root:    Street edge: 10637
DEBUG:ro

DEBUG:root:    Street edge: 20479
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 25872
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 16588
DEBUG:root:      Num interactions: 1699
DEBUG:root:    Street edge: 25874
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 16597
DEBUG:root:      Num interactions: 5462
DEBUG:root:    Street edge: 14514
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 16610
DEBUG:root:      Num interactions: 5646
DEBUG:root:    Street edge: 2025
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 16633
DEBUG:root:      Num interactions: 2199
DEBUG:root:    Street edge: 18798
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 16641
DEBUG:root:      Num interactions: 2139
DEBUG:root:Condition_id, worker_id: 100, AMDX0UNZS4A1G
DEBUG:root:Assignment id: 774
DEBUG:root:  Route_id: 205
DEBUG:root:    Street edge: 19873
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit 

...

DEBUG:root:      Num interactions: 18847
DEBUG:root:  Route_id: 111
DEBUG:root:    Street edge: 615
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 21609
DEBUG:root:      Num interactions: 18847
DEBUG:root:    Street edge: 23125
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:  Route_id: 109
DEBUG:root:    Street edge: 12339
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 12340
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 12341
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 17138
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:Condition_id, worker_id: 92, A23YZ4U8UD4J6F
DEBUG:root:Assignment id: 905
DEBUG:root:  Route_id: 68
DEBUG:root:    Street edge: 11373
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 15583
DEBUG:root:      Num interactions: 3323
DEBUG:root:    Street edge: 10637
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 11377
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:  Route_id: 69
D

DEBUG:root:    Street edge: 2025
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 18798
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:Condition_id, worker_id: 100, A1SOOCPS9CRCQE
DEBUG:root:Assignment id: 748
DEBUG:root:  Route_id: 205
DEBUG:root:    Street edge: 19873
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 15149
DEBUG:root:      Num interactions: 12727
DEBUG:root:    Street edge: 16929
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 15232
DEBUG:root:      Num interactions: 2948
DEBUG:root:  Route_id: 206
DEBUG:root:    Street edge: 16929
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 15232
DEBUG:root:      Num interactions: 2948
DEBUG:root:    Street edge: 16932
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 16243
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 25926
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:  Route_id: 207
DEBUG:root:    Street edge: 16923
DEBUG:root:    Num au

DEBUG:root:Condition_id, worker_id: 108, A1HRHFU7KTS0KW
DEBUG:root:Assignment id: 3003
DEBUG:root:  Route_id: 215
DEBUG:root:    Street edge: 692
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 26097
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 20896
DEBUG:root:      Num interactions: 15739
DEBUG:root:    Street edge: 3133
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 20923
DEBUG:root:      Num interactions: 18779
DEBUG:root:    Street edge: 24572
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 20946
DEBUG:root:      Num interactions: 18309
DEBUG:root:  Route_id: 216
DEBUG:root:    Street edge: 24572
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 20946
DEBUG:root:      Num interactions: 18309
DEBUG:root:    Street edge: 11029
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 5892
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 22898
DEBUG:root:    Num audit_tasks: 0
DEB

DEBUG:root:    Street edge: 22057
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:Condition_id, worker_id: 113, A2U97DA5GDRRML
DEBUG:root:Assignment id: 2299
DEBUG:root:  Route_id: 8
DEBUG:root:    Street edge: 25551
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 25552
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 10553
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 2060
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 24414
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 24413
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:  Route_id: 7
DEBUG:root:    Street edge: 3735
DEBUG:root:    Num audit_tasks: 1
DEBUG:root:      Audit task id: 18784
DEBUG:root:      Num interactions: 7
DEBUG:root:    Street edge: 25551
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:  Route_id: 9
DEBUG:root:    Street edge: 24413
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 15089
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    St

KeyboardInterrupt: 

In [ ]:
features

In [ ]:
features.describe()

In [ ]:
len(selected_users)

In [12]:
d = get_features(86, 'A1Y55YV1WA1TTD')

DEBUG:root:Condition_id, worker_id: 86, A1Y55YV1WA1TTD
DEBUG:root:Assignment id: 2873
DEBUG:root:  Route_id: 199
DEBUG:root:    Street edge: 23231
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 23299
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 14467
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 14465
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 1059
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 16899
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:  Route_id: 201
DEBUG:root:    Street edge: 5589
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 15767
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 23230
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 23227
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 23225
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 21961
DEBUG:root:    Num audit_tasks: 0
DEBUG:root:    Street edge: 24235
DEBUG:root: 

In [ ]:
l = turker_accuracy
l = l[l['binary'] == True]
l = l[l['granularity'] == '5_meter']
l = l[l['remove.low.severity'] == False]
l = l[l['n.workers'] == 1]
labels = l[['condition.id', 'worker1', 'label.type', 'precision', 'recall', 'specificity', 'f.measure']]
labels.reset_index(inplace=True)
labels.head()

In [ ]:
data = labels.merge(features, left_on=['condition.id', 'worker1'], right_on=['condition_id', 'worker_id'])
data.head()

In [ ]:
data.to_csv('interaction_features.csv')

In [ ]:
len(selected_users)

In [ ]:
len(turker_accuracy[['condition.id', 'worker1']].drop_duplicates())

In [ ]:
d = pd.read_csv('interaction_features.csv')

In [ ]:
len(d[['condition.id', 'worker1']].drop_duplicates())